# ETAPA 8 — Desenvolvimento da API com FastAPI

## Objetivo desta etapa

Nesta etapa, iniciaremos o desenvolvimento de uma API RESTful em FastAPI, que será responsável por servir o modelo de classificação de crédito treinado e versionado previamente com MLflow. Essa API será o componente intermediário entre o frontend (Streamlit) e os artefatos de modelo armazenados no backend (MinIO via DVC).

A API será desenvolvida dentro de um novo notebook técnico, seguindo o protocolo de engenharia progressiva em ambiente containerizado. O código posteriormente será consolidado em um ou mais arquivos `.py` para versionamento estável no repositório Git.

## Componentes que serão implementados

A arquitetura da API inclui:

1. Endpoint principal `/predict` (POST)  
   - Recebe os dados do cliente via JSON.  
   - Aplica o pré-processamento necessário (OrdinalEncoder treinado).  
   - Carrega o modelo Random Forest Tuned a partir do repositório MLflow (usando o `run_id` consolidado).  
   - Realiza a inferência e retorna a classe predita (`Good`, `Standard`, `Poor`).

2. Autenticação por API Key  
   - Chave secreta validada no cabeçalho da requisição (`X-API-Key`).  
   - Requisições não autenticadas serão rejeitadas com HTTP 401.

3. Throttling via `slowapi`  
   - Limita o número de requisições por IP para evitar abuso.  
   - Exemplo: máximo 10 requisições por minuto.

4. Middlewares auxiliares  
   - CORS configurado (caso o frontend esteja hospedado em outro host).  
   - Logger básico das requisições e respostas.

## Tecnologias e bibliotecas

- `fastapi` — framework principal da API  
- `uvicorn` — servidor ASGI leve para execução local  
- `pydantic` — definição do schema de entrada (validação automática)  
- `mlflow.sklearn` — para carregar o modelo versionado com rastreamento  
- `slowapi` — controle de taxa de requisições (throttling)  
- `python-dotenv` — carregamento seguro da chave da API via `.env`

## Processo de desenvolvimento

O desenvolvimento seguirá estas fases no notebook:

1. Definição do schema do input via Pydantic.  
2. Carregamento do modelo RandomForestClassifier treinado com MLflow.  
3. Criação da função de pré-processamento com o mesmo OrdinalEncoder usado no treino.  
4. Construção do app FastAPI com endpoint `/predict` e autenticação.  
5. Testes locais com `requests` para simular chamadas reais.  
6. Criação dos arquivos finais `api.py` e `.env.example` para deploy.

## Observações finais

- O modelo que será servido já está salvo em `runs:/4e56a5afe29a4a26b962c220fef03f5d/random_forest_tuned_model`.  
- A API será testada localmente no DevContainer via `uvicorn`.  
- Todo o desenvolvimento será realizado de forma incremental, validando cada componente separadamente antes da integração completa.




# ETAPA: Definição do schema de entrada para o endpoint /predict

Nesta célula, definimos o schema de entrada que será usado pela API FastAPI para validar os dados recebidos no endpoint `/predict`.  
Utilizamos o `pydantic.BaseModel` para garantir que todas as features esperadas estejam presentes e no formato correto.

Esse schema deve refletir exatamente as colunas necessárias para o modelo `RandomForestClassifier` treinado, com exceção do target (`Credit_Score`), que não será fornecido no input.

A validação será automática: se algum campo estiver ausente ou mal formatado, a API responderá com erro HTTP 422.

As variáveis esperadas correspondem ao conjunto `X_test` no pipeline `curated_v1_1`, com os mesmos nomes e tipos.


In [1]:
# ETAPA: Definição do schema de entrada para o endpoint /predict

from pydantic import BaseModel
from typing import Literal, Optional

class InputData(BaseModel):
    Age_Binned: int
    Amount_invested_monthly_Binned: int
    Annual_Income_Binned: int
    Changed_Credit_Limit_Binned: int
    Credit_History_Age: float
    Credit_History_Age_Binned: int
    Credit_Mix: int
    Credit_Utilization_Ratio_Binned: int
    Delay_from_due_date_Binned: int
    Interest_Rate_Binned: int
    Monthly_Balance_Binned: int
    Monthly_Inhand_Salary_Binned: int
    Num_Bank_Accounts_Binned: int
    Num_Credit_Card_Binned: int
    Num_Credit_Inquiries_Binned: int
    Num_of_Delayed_Payment_Binned: int
    Num_of_Loan_Binned: int
    Occupation: int
    Outstanding_Debt_Binned: int
    Payment_of_Min_Amount: int
    Total_EMI_per_month_Binned: int
    Type_of_Loan: int


# ETAPA: Carregamento do modelo Random Forest Tuned e pré-processamento com OrdinalEncoder

Nesta célula, implementamos os dois componentes fundamentais que serão reutilizados pela API:

1. Carregamento do modelo `RandomForestClassifier` salvo no MLflow (via `run_id` fixado);
2. Reaplicação do `OrdinalEncoder`, treinado sobre as mesmas colunas categóricas utilizadas na etapa de modelagem.

Ambos os passos devem ocorrer **fora da rota** da API (no escopo global do `api.py`), para que o modelo e o encoder sejam carregados **uma única vez** no momento em que o servidor for iniciado, garantindo desempenho na inferência.

O `OrdinalEncoder` é treinado com o mesmo conjunto `train_curated_v1_1.csv` para manter a consistência do encoding.

O `model_uri` e o caminho do CSV devem ser ajustáveis em variáveis fixas no início do script.


In [2]:
# ETAPA: Carregamento do modelo Random Forest Tuned e pré-processamento com OrdinalEncoder

import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.preprocessing import OrdinalEncoder

# Define o tracking URI para acesso local ao .mlruns
mlflow.set_tracking_uri("file:/workspace/.mlruns")

# Caminhos fixos
MODEL_RUN_ID = "4e56a5afe29a4a26b962c220fef03f5d"
MODEL_URI = f"runs:/{MODEL_RUN_ID}/random_forest_tuned_model"
TRAIN_CSV = "/workspace/data/curated/train_curated_v1_1.csv"

# Carrega o modelo uma única vez
model = mlflow.sklearn.load_model(MODEL_URI)
print("Modelo carregado com sucesso.")

# Carrega o treino e extrai colunas categóricas
df_train = pd.read_csv(TRAIN_CSV)
X_train = df_train.drop(columns=["Credit_Score"])
object_cols = X_train.select_dtypes(include="object").columns.tolist()

# Treina o encoder nas colunas categóricas
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
encoder.fit(X_train[object_cols])
print(f"OrdinalEncoder ajustado nas colunas: {object_cols}")


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Modelo carregado com sucesso.
OrdinalEncoder ajustado nas colunas: ['Age_Binned', 'Amount_invested_monthly_Binned', 'Annual_Income_Binned', 'Changed_Credit_Limit_Binned', 'Credit_History_Age', 'Credit_History_Age_Binned', 'Credit_Mix', 'Credit_Utilization_Ratio_Binned', 'Delay_from_due_date_Binned', 'Interest_Rate_Binned', 'Monthly_Balance_Binned', 'Monthly_Inhand_Salary_Binned', 'Num_Bank_Accounts_Binned', 'Num_Credit_Card_Binned', 'Num_Credit_Inquiries_Binned', 'Num_of_Delayed_Payment_Binned', 'Num_of_Loan_Binned', 'Occupation', 'Outstanding_Debt_Binned', 'Payment_of_Min_Amount', 'Total_EMI_per_month_Binned', 'Type_of_Loan']


# ETAPA: Criação do app FastAPI com rota /predict e autenticação por API Key

Nesta célula, criamos a estrutura do app FastAPI com os seguintes recursos:

- **Rota `/predict` (POST):** recebe um objeto JSON no formato validado por `InputData`, aplica o `OrdinalEncoder` e realiza a inferência usando o modelo Random Forest já carregado.

- **Autenticação com API Key:** a chave será esperada no cabeçalho `X-API-Key`. A chave real será lida de uma variável de ambiente (`API_KEY`), recomendando-se uso de `.env`.

- **Resposta da rota:** a classe predita é retornada como texto (`Good`, `Standard` ou `Poor`), convertendo de valor numérico (0, 1, 2) para rótulo humano.

Essa estrutura permite que apenas usuários autenticados façam chamadas ao modelo, preparando a aplicação para uso em ambiente controlado.

A definição da chave (`API_KEY`) deve ser feita no arquivo `.env` ou via `os.environ`.


In [3]:
# ETAPA: Criação do app FastAPI com rota /predict e autenticação por API Key

from fastapi import FastAPI, Header, HTTPException
from pydantic import BaseModel
import os

# Classe para validar input (deve estar no mesmo arquivo ou importada)
from typing import Literal
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

# Carrega API_KEY do ambiente
API_KEY = os.environ.get("API_KEY", "default_secret")  # Substituir em produção

# Middleware opcional para permitir chamadas locais (ex: Streamlit em outra porta)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Pode ser restrito depois
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Mapeia número para rótulo
label_map = {0: "Poor", 1: "Standard", 2: "Good"}

@app.post("/predict")
def predict(data: InputData, x_api_key: str = Header(...)):

    if x_api_key != API_KEY:
        raise HTTPException(status_code=401, detail="Invalid API Key")

    # Converte o input em DataFrame
    input_df = pd.DataFrame([data.dict()])

    # Aplica o encoder nas colunas categóricas
    input_df[object_cols] = encoder.transform(input_df[object_cols])

    # Faz a predição
    prediction = model.predict(input_df)[0]
    label = label_map.get(prediction, "Unknown")

    return {"prediction": label}


# Etapa: Execução Local do Servidor FastAPI com `uvicorn` a partir do arquivo `api.py`

Este bloco técnico realiza a execução do servidor FastAPI utilizando o módulo `uvicorn`, com o objetivo de validar o endpoint `/predict` localmente na porta 8000.

A estrutura da API já foi definida anteriormente, com autenticação via API Key, aplicação de encoder e carregamento do modelo. O arquivo `api.py` encontra-se fixado na raiz do diretório `/workspace/`, conforme decisão técnica consolidada.

A execução será feita em modo persistente (sem `reload`), assumindo que:

- A porta 8000 esteja disponível.
- O ambiente já tenha os modelos carregados corretamente no `api.py`.
- A variável de ambiente `API_KEY` esteja configurada para autenticação.

Se houver ocupação da porta 8000, o processo atual será identificado e encerrado de forma controlada antes da nova execução.


In [6]:
# ETAPA: Execução local do servidor FastAPI com Uvicorn a partir de /workspace/api.py

import subprocess
import os
import signal

# 1️⃣ Identifica se já existe processo escutando na porta 8000
try:
    pid_check = subprocess.run(["lsof", "-t", "-i:8000"], capture_output=True, text=True)
    pid = pid_check.stdout.strip()

    if pid:
        print(f"Processo detectado na porta 8000 (PID {pid}). Encerrando...")
        os.kill(int(pid), signal.SIGTERM)
        print("✔️ Processo encerrado.")
    else:
        print("✔️ Porta 8000 está livre.")

except FileNotFoundError:
    print("⚠️ Comando 'lsof' não disponível. Recomendado instalar via apt install lsof.")

# 2️⃣ Executa o servidor uvicorn diretamente no workspace
print("Iniciando uvicorn com api.py localizado em /workspace...")

# Executa uvicorn forçando o diretório correto como raiz
result = subprocess.run([
    "uvicorn",
    "api:app",
    "--host", "0.0.0.0",
    "--port", "8000"
], cwd="/workspace")

print(f"\n🔚 uvicorn finalizado com código: {result.returncode}")

✔️ Porta 8000 está livre.
Iniciando uvicorn com api.py localizado em /workspace...


INFO:     Started server process [155723]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [155723]


KeyboardInterrupt: 